# Entorno Remoto de Compilación (MambaVision) 🚀

Este notebook se ejecuta enteramente en los servidores de Google Colab conectado desde tu VS Code.
Sigue los pasos secuenciales para descargar tu código desde GitHub y compilar el Kernel Custom de C++ contra los headers de TensorFlow provistos por Google.

In [4]:
import os
import sys

# ==========================================
# CONFIGURACIÓN DE TU REPOSITORIO DE GITHUB
# ==========================================
GITHUB_REPO = "https://github.com/rdromer2/mambavision_tf_kernel.git"
REPO_NAME = "mambavision_tf_kernel"
PROJECT_PATH = f"/content/{REPO_NAME}"

# 1. Clonar o actualizar el repositorio
if not os.path.exists(PROJECT_PATH):
    print(f"Clonando el repositorio {GITHUB_REPO}...")
    os.system(f'git clone {GITHUB_REPO} {PROJECT_PATH}')
else:
    print("El repositorio ya existe. Actualizando con git pull...")
    os.system(f'cd {PROJECT_PATH} && git pull')

# 2. Añadir al sys.path
sys.path.append(PROJECT_PATH)

# 3. Cambiar nuestro directorio activo al proyecto
os.chdir(PROJECT_PATH)
print(f"\nDirectorio de trabajo actual: {os.getcwd()}")

El repositorio ya existe. Actualizando con git pull...

Directorio de trabajo actual: /content/mambavision_tf_kernel


## 🛠️ Compilación C++ delegada en CMake
Delegamos integramente el proceso de construcción en CMake en la subcarpeta `build/`, lo que garantiza mayor mantenibilidad y asegura que las aserciones formales y el fix de ABI establecidos en `CMakeLists.txt` surtan efecto sistemáticamente.

In [5]:
import os

print("Limpiando compilaciones anteriores...")
# El comando de purga se inyecta aquí, antes de crear la carpeta de nuevo
!rm -rf {PROJECT_PATH}/build

print("Configurando y Compilando el Kernel C++ y CUDA...")
!mkdir -p {PROJECT_PATH}/build
%cd {PROJECT_PATH}/build
!cmake ..
!make -j
%cd {PROJECT_PATH}

print("\n✅ Compilación finalizada.")

Limpiando compilaciones anteriores...
Configurando y Compilando el Kernel C++ y CUDA...
/content/mambavision_tf_kernel/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- The CUDA compiler identification is NVIDIA 12.8.93 with host compiler GNU 11.4.0
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Found Python3: /usr/local/bin/python (found version "3.1

## 🧪 Prueba Inicial del Kernel (El Centinela de Memoria)
**Detalle Técnico**: Mantenemos el hack de `ctypes.RTLD_GLOBAL` vital porque seguimos utilizando Colab y los binarios de TF no están compilados estáticamente.
Esta prueba evaluará un flujo correcto (3D) y un caso de fallo intencionado (2D) para verificar la intercepción del centinela de memoria.

In [ ]:
import tensorflow as tf
import numpy as np

# 1. Cargar la librería compartida compilada
mamba_module = tf.load_op_library('./libmamba_ssm.so')

# 2. Configurar dimensiones de prueba
batch_size = 1
seq_len = 4
d_model = 8

# 3. Crear tensor de entrada aleatorio
np.random.seed(42)
u_in_np = np.random.rand(batch_size, seq_len, d_model).astype(np.float32)
u_in_tf = tf.convert_to_tensor(u_in_np)

# 4. Ejecutar el Kernel de CUDA a través de TensorFlow
with tf.device('/GPU:0'):
    out_tf = mamba_module.mamba_selective_scan(u_in_tf)

# 5. Implementación de Referencia en NumPy (Para validar la matemática)
# Marcadores estáticos usados en el kernel CUDA
delta = 0.1
A = -1.0
B = 1.0

A_bar = np.exp(A * delta)
B_bar = B * delta

out_ref_np = np.zeros_like(u_in_np)

# Simular la propagación Hilo-por-Canal en Python
for b in range(batch_size):
    for d in range(d_model):
        h_prev = 0.0
        for t in range(seq_len):
            x_t = u_in_np[b, t, d]
            h_prev = A_bar * h_prev + B_bar * x_t
            out_ref_np[b, t, d] = h_prev

# 6. Comparar resultados
out_tf_np = out_tf.numpy()

print("Salida TensorFlow (CUDA):")
print(out_tf_np[0, :, 0]) # Imprimir el primer canal a lo largo del tiempo
print("\nSalida Referencia (NumPy):")
print(out_ref_np[0, :, 0])

# Validación estricta con np.allclose (tolerancia para punto flotante de 32 bits)
is_correct = np.allclose(out_tf_np, out_ref_np, atol=1e-5, rtol=1e-5)

print(f"\n¿Coincide la matemática de CUDA con la referencia? {is_correct}")

Precargando RTLD_GLOBAL: /usr/local/lib/python3.12/dist-packages/tensorflow/libtensorflow_framework.so.2

Iniciando transferencia a VRAM y ejecución CUDA...
Tensor entrada (muestra): [1. 1. 1. 1. 1.]
Tensor salida  (muestra): [2. 2. 2. 2. 2.]

Estado de validación matemática (x * 2.0): True
